# Local averages

In this hands-on your task is to optimize the performance of a kernel that computes averages.
The input is a one-dimensional array of size **N**, and the input is a different one-dimensional array of size **N/4** where each element **i** is the average of 4 consecutive elements of the input array.

Do not worry if the definition at this stage is still a bit vague, the code will be soon presented and you will realize it is self explanatory.
But first, let us start by importing the necessary Python modules, initialize the GPU, and create the necessary arrays.

In this notebook we will be using Kernel Tuner, if you haven't installed Kernel Tuner yet, please run the following cell.

In [ ]:
%pip install kernel_tuner

Now we are ready to import the necessary modules

In [ ]:
from collections import OrderedDict
import numpy as np
import kernel_tuner

Let's create the input and output data using Numpy

In [ ]:
N = np.int32(10e6)
A = np.random.randn(N).astype(np.float32)
B1 = np.zeros(N//4).astype(np.float32)
B2 = np.zeros_like(B1)

Now that we have the right data structures, we can write a naive function to compute our local averages. This function is intentionally written in a C-like programming style, to help students that are less experienced with Python.

In [ ]:
def local_averages(A, B, N):
    for i in range(0, np.int32(N/4)):
        temp = 0.0
        for j in range(0, 4):
            temp = temp + A[(i * 4) + j]
        B[i] = temp / 4.0

We can now execute and time our code. In this way we will save our reference output (for testing purpose) and have a glimpse at the execution time on the CPU.

In [ ]:
%%time

local_averages(A, B1, N)

A slightly more pythonic and much faster version of this function using Numpy is:

In [ ]:
%%time

ref_b = np.average(A.reshape(N//4, 4), axis=1)

Let's make sure these compute the same:

In [ ]:
print("PASSED" if np.allclose(ref_b, B1, atol=1e-6) else "FAILED")

It is now time to introduce the naive CUDA code, and save it to a local file, as done in any of the previous exercises. The main difference this time is that the code is already complete and correct.

In [ ]:
%%writefile local_averages.cu

__global__ void local_averages_kernel(float * A, float * B, int size_B)
{
    int index = (blockIdx.x * blockDim.x) + threadIdx.x;

    if ( index < size_B )
    {
        float temp = 0.0;

        for ( int j = 0; j < 4; j++ )
        {
            temp = temp + A[(index * 4) + j];
        }
        B[index] = temp / 4.0;
    }
}

Your goal at this point is to understand how this kernel works, and improve its performance. But before doing that, let us use Kernel Tuner to measure the performance of this kernel.

In [ ]:
#we will specify the tunable parameters of this kernel using a dictionary
tune_params = OrderedDict()

#using the special name "block_size_x" we can specify what values
#Kernel Tuner should use for the number of threads per block in the x-dimension
tune_params["block_size_x"] = [1024]

#we can also specify how Kernel Tuner should compute performance metrics
metrics = OrderedDict(GFLOPs=lambda p: (N/4*5/1e9)/(p["time"]/1e3))

res, env = kernel_tuner.tune_kernel("local_averages_kernel", #the name of the kernel
                                    "local_averages.cu",     #kernel source file
                                    N//4,                    #problem size
                                    [A, B2, N],              #kernel argument list
                                    tune_params,             #tunable parameters
                                    answer=[None, B1, None], #reference answer
                                    metrics=metrics,         #performance metric
                                    lang="cupy")             #select cupy backend

Executing the above cell gave us our starting point for our optimization process, which is the execution time of our naive kernel.

It is now your turn to change the CUDA code (don't forget to run the cell to write your changes to file) and improve the performance of the kernel.

To avoid you losing track of the naive kernel's execution time, we are going to replicate the previous cell below this one. Just go back to the cell containing the CUDA code, modify the code, run that cell, and then run the one below. Because we use the ``answer`` option of tune_kernel, Kernel Tuner will complain if you make changes that invalidate the correctness of the kernel's output.

In [ ]:
#we will specify the tunable parameters of this kernel using a dictionary
tune_params = OrderedDict()

#using the special name "block_size_x" we can specify what values
#Kernel Tuner should use for the number of threads per block in the x-dimension
tune_params["block_size_x"] = [1024]

#we can also specify how Kernel Tuner should compute performance metrics
metrics = OrderedDict(GFLOPs=lambda p: (N/4*5/1e9)/(p["time"]/1e3))

res, env = kernel_tuner.tune_kernel("local_averages_kernel", #the name of the kernel
                                    "local_averages.cu",     #kernel source file
                                    N//4,                    #problem size
                                    [A, B2, N],              #kernel argument list
                                    tune_params,             #tunable parameters
                                    answer=[None, B1, None], #reference answer
                                    metrics=metrics,         #performance metric
                                    lang="cupy")             #select cupy backend